In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import transforms, datasets, models
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import os
from time import time
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"The process is running in {device}")

The process is running in cpu


In [12]:
# Transform the dataset

data_transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
}

In [15]:
# dataset

data_dir = 'data'
image_dataset = {
    'train': datasets.ImageFolder(os.path.join(data_dir,'train'), transform= data_transform['train']),
    'val' : datasets.ImageFolder(os.path.join(data_dir,'test'), transform= data_transform['test'])
}

In [16]:
data_loader = {
            "train": DataLoader(image_dataset["train"], batch_size=16, shuffle=True, num_workers=4),
            "val": DataLoader(image_dataset["val"], batch_size=16, shuffle=True, num_workers=4)
        }

In [8]:
for x, y in data_loader['train']:
    print(x.shape)
    print(y.shape)
    break

torch.Size([16, 3, 640, 640])
torch.Size([16])


/Users/shyam/anaconda3/envs/pytorch_image_classification/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/Users/shyam/anaconda3/envs/pytorch_image_classification/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fba56a4f1c0>
Traceback (most recent call last):
  File "/Users/shyam/anaconda3/envs/pytorch_image_classification/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/Users/shyam/anaconda3/envs/pytorch_image_classification/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/shyam/anaconda3/envs/pytorch_

In [17]:
model = models.get_model("resnet18", pretrained=True)

classes = image_dataset["train"].classes

# Freeze the layers
for param in model.parameters():
    param.required_grad = False

no_of_features = model.fc.in_features
model.fc = nn.Linear(no_of_features, len(classes)) #incoming and outgoing features
model = model.to(device)


In [18]:
epochs = 20
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
for epoch in range(epochs):
    print(f"Epoch {epoch}/{epochs-1}")
    print('*'*50)

    for phase in ["train", "val"]:
        training = phase == "train"

        if training:
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_accuracy = 0.0
        print(data_loader[phase])
        with tqdm(data_loader[phase]) as tepoch:
            for x,y in tepoch:
                x = x.to(device)
                y = y.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(training):
                    y_pred = model(x)
                    loss = loss_fn(y_pred, y)
                    
                    if training:
                        loss.backward()
                        optimizer.step()
                    
                    # Calculate the accuracy
                    train_pred = torch.max(y_pred, dim=1).indices
                    running_loss += loss.item() * x.size(0)
                    running_accuracy += torch.sum(train_pred == y.data)

            epoch_loss = running_loss / len(data_loader[phase].dataset) #len(data_loader[phase])             
            epoch_accuracy = running_accuracy.double() / len(data_loader[phase].dataset) #len(data_loader[phase])

            print(f"Epoch: {epoch}, {phase} Loss: {epoch_loss}, Accuracy: {epoch_accuracy}")


Epoch 0/19
**************************************************


TypeError: 'module' object is not callable

In [19]:
def train_loop(model, data_loader, loss_fn, optimizer, epochs):

    for epoch in range(epochs):
        print(f"Epoch {epoch}/{epochs-1}")
        print('*'*50)

        for phase in ["train", "test"]:
            training = phase == "train"

            if training:
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_accuracy = 0.0

            with tqdm(data_loader[phase]) as tepoch:
                for x,y in tepoch:
                    x = x.to(device)
                    y = y.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(training):
                        y_pred = model(x)
                        loss = loss_fn(y_pred, y)
                        
                        if training:
                            loss.backward()
                            optimizer.step()
                        
                        # Calculate the accuracy
                        train_pred = torch.max(y_pred, dim=1).indices
                        running_loss += loss.item() * x.size(0)
                        running_accuracy += torch.sum(train_pred == y.data)

                epoch_loss = running_loss / len(data_loader[phase].dataset) #len(data_loader[phase])             
                epoch_accuracy = running_accuracy.double() / len(data_loader[phase].dataset) #len(data_loader[phase])

                print(f"Epoch: {epoch}, {phase} Loss: {epoch_loss}, Accuracy: {epoch_accuracy}")

    return model

model = train_loop(model, data_loader, loss_fn, optimizer, 20)

Epoch 0/19
**************************************************


TypeError: 'module' object is not callable